In [1]:
import open3d
import numpy as np
import pandas as pd
import os
import glob
import tqdm
import copy
import matplotlib.pyplot as plt

from time import sleep
from scipy.signal import argrelmin, argrelmax
from PIL import Image

import utils.registration as registration
import utils.fread as fread
import utils.FCGF as FCGF
import utils.pointcloud as pointcloud
import utils.transform as transform
import utils.grid_search as grid_search

from utils.config import Config
from utils.functions import *

In [2]:
def calc_overlap(config: Config):
    if not os.path.exists(config.get_output_file(f"{config.get_file_name()}.npz")):
        print("Unable to find trajectory data. Skipping.")
        return
    
    if not os.path.exists(os.path.join(config.get_groundtruth_dir(), f"{config.get_file_name()}.gtpose.npz")):
        print("Unable to find groundtruth data. Skipping.")
        return
    
    if os.path.exists(config.get_output_file(f"{config.get_file_name()}__overlap.csv")):
        print("Overlap already calculated. Skipping.")
        return
    
    groundtruth_data = np.load(os.path.join(config.get_groundtruth_dir(), f"{config.get_file_name()}.gtpose.npz"))
    estimated_data = np.load(config.get_output_file(f"{config.get_file_name()}.npz"))
    
    feature_dir = config.get_feature_dir()
    sequence_ts = fread.get_timstamps(feature_dir, ext=".secondary.npz")
    sequence_ts = fread.sample_timestamps(sequence_ts, config.target_fps)
    
    target_ts = estimated_data["sequence_ts"]

    target_inds = [np.argwhere(sequence_ts == target_ts[i])[0][0] for i in range(len(target_ts))]
    
    local_t = groundtruth_data["local_t"][target_inds]

    sequence_ts = sequence_ts[target_inds]
    num_frames = len(sequence_ts)

    std_values = []

    for t in tqdm.trange(len(sequence_ts)):
        depth_img_file = os.path.join(config.get_sequence_dir(), f"frame-{sequence_ts[t]}.depth.png")
        std_values.append(registration.calc_std(depth_img_file, 4000))
        
    std_values = np.array(std_values)

    local_pcds = []

    for t in tqdm.trange(num_frames):
        feature_file = os.path.join(config.get_feature_dir(), f"{sequence_ts[t]}.secondary.npz")
        pcd = FCGF.get_features(feature_file, config.voxel_size, pcd_only=True)
        local_pcds.append(pcd)
        

    device_0_ts = fread.get_timstamps_from_images(os.path.join(config.get_global_dir(), "device-0"), ext=".depth.png")
    device_1_ts = fread.get_timstamps_from_images(os.path.join(config.get_global_dir(), "device-1"), ext=".depth.png")
    device_2_ts = fread.get_timstamps_from_images(os.path.join(config.get_global_dir(), "device-2"), ext=".depth.png")

    y = [[], [], []]

    for i in range(num_frames):
        y[0].append(nearest(device_0_ts, sequence_ts[i]))
        y[1].append(nearest(device_1_ts, sequence_ts[i]))
        y[2].append(nearest(device_2_ts, sequence_ts[i]))
        
        y[0][i] = np.abs(y[0][i] - sequence_ts[i]) * 1e-6
        y[1][i] = np.abs(y[1][i] - sequence_ts[i]) * 1e-6
        y[2][i] = np.abs(y[2][i] - sequence_ts[i]) * 1e-6
        
    print("-- Finding optimal global positions for registration")

    global_frame_delays = np.array(y)
    global_frame_delays_inds = np.ones(global_frame_delays.shape, dtype=np.int8)

    for r, c in np.argwhere(global_frame_delays > 100):
        global_frame_delays_inds[r, c] = 0
        
    global_frame_delays_inds = np.sum(global_frame_delays_inds, axis=0)
    global_frame_delays_inds = np.where(global_frame_delays_inds == 3, 1, 0)
        
    plt.figure(figsize=(20, 6))
    plt.plot(global_frame_delays_inds)
    plt.xlabel("Frame #")
    plt.ylabel("Availability of Global Frame")

    plt.savefig(config.get_output_file(f"{config.get_file_name()}__availability.jpeg"))
    plt.close()

    global_pos = [0]
    for t in tqdm.trange(num_frames):
        if t - global_pos[-1] >= config.target_fps * 0.8: 
            global_pos.append(t)
            continue
        
        if (np.abs(std_values[t] - std_values[global_pos[-1]]) > config.delta) and (t - global_pos[-1] > config.target_fps * 0.5):
            global_pos.append(t)

    global_pos = np.array(global_pos)

    cutoffs = registration.find_cutoffs(std_values, config.target_fps, config.min_std, config.threshold)
    
    plt.figure(figsize=(20, 6))
    plt.plot(std_values)

    plt.scatter(global_pos, std_values[global_pos], c="r", marker="+")

    for x in cutoffs:
        plt.axvline(x - config.cutoff_margin, c="g", linestyle="--")
        plt.axvline(x, c="b", linestyle="--")
        plt.axvline(x + config.cutoff_margin, c="g", linestyle="--")

    plt.axhline(y=config.min_std, color="r", linestyle="--")
    plt.ylim(0, 4)
    plt.xlim(0, len(std_values))

    plt.xlabel("Frame #")
    plt.ylabel("Std. of Distances to the camera")

    plt.savefig(config.get_output_file(f"{config.get_file_name()}__std.jpeg"))
    plt.close()

    cutoffs = registration.get_cutoff_sequence(std_values, config.target_fps, config.min_std, config.threshold, config.cutoff_margin)
    
    target = FCGF.get_features("data/reference/larc_kitchen_v5.npz", config.voxel_size, pcd_only=True)

    # plt.figure(figsize=(20, 10))
    overlap = []

    for i in range(len(cutoffs)):

        start_t, end_t = cutoffs[i]
        global_inds = global_pos[np.logical_and(global_pos >= start_t, global_pos <= end_t)]
        
        availability = np.sum(global_frame_delays_inds[start_t:end_t]) / (end_t - start_t)
        
        fragment = []

        for t in range(start_t, end_t):
            local_temp = copy.deepcopy(local_pcds[t])
            local_temp.transform(local_t[t])
            fragment.append(local_temp)
            
        fragment = pointcloud.merge_pcds(fragment, 0.03)
        fragment.paint_uniform_color([0.5, 0.5, 0.5])

        result = open3d.registration.evaluate_registration(fragment, target, 0.05, np.identity(4))
        
        overlap.append([start_t, end_t, result.fitness, len(global_inds) >= 3, availability])
        
        # gt_points = np.asarray(target.points)
        # gt_x, gt_y = gt_points[:, 0], gt_points[:, 2]
        
        # fg_points = np.asarray(fragment.points)
        # fg_x, fg_y = fg_points[:, 0], fg_points[:, 2]
        
        # plt.subplot(1, len(cutoffs), i + 1)
        # plt.scatter(gt_x, gt_y, s=0.1, c="r")
        # plt.scatter(fg_x, fg_y, s=0.1, c="b")
        # plt.xlim(-4.5, 4)
        # plt.ylim(-5.5, 4.5)
        # plt.gca().set_aspect('equal', adjustable='box')
        # plt.axis("off")
        # plt.title(f"({start_t}, {end_t}) : {result.fitness:.2f}, {len(global_inds) >= 3}")

    df = pd.DataFrame(overlap, columns=["start_t", "end_t", "overlap", "has_enough_global_frames", "availability"])
    df.to_csv(config.get_output_file(f"{config.get_file_name()}__overlap.csv"), index=False)

    # plt.savefig(config.get_output_file(f"{config.get_file_name()}__overlap.jpeg"))
    # plt.close()

In [3]:
config = Config(
        sequence_dir="data/raw_data",
        feature_dir="data/features",
        output_dir="data/trajectories/trajectory/IMU_PCD_outlier_removed_0.05",
        experiment="exp_12",
        trial="trial_1",
        subject="subject-1",
        sequence="01",
        groundtruth_dir="data/trajectories/groundtruth",
    )
    
config.voxel_size=0.05
config.target_fps=20
config.min_std=0.5

In [4]:
for trial in os.listdir(os.path.join(config.feature_dir, config.experiment)):
    config.trial = trial
    for subject in os.listdir(os.path.join(config.feature_dir, config.experiment, config.trial, str(config.voxel_size))):
        config.subject = subject    
        for sequence in os.listdir(os.path.join(config.feature_dir, config.experiment, config.trial, str(config.voxel_size), config.subject)):
            config.sequence = sequence
            print(f"Processing: {config.experiment} >> {config.trial} >> {config.subject} >> {config.sequence}")
            calc_overlap(config)

Processing: exp_12 >> trial_1 >> subject-1 >> 01
Overlap already calculated. Skipping.
Processing: exp_12 >> trial_1 >> subject-1 >> 02
Overlap already calculated. Skipping.
Processing: exp_12 >> trial_1 >> subject-1 >> 03
Overlap already calculated. Skipping.
Processing: exp_12 >> trial_1 >> subject-1 >> 04
Overlap already calculated. Skipping.
Processing: exp_12 >> trial_1 >> subject-1 >> 05
Overlap already calculated. Skipping.
Processing: exp_12 >> trial_2 >> subject-1 >> 01
Overlap already calculated. Skipping.
Processing: exp_12 >> trial_2 >> subject-1 >> 02


100%|██████████| 417/417 [00:06<00:00, 63.54it/s]


-- Finding optimal global positions for registration


100%|██████████| 417/417 [00:00<00:00, 285229.09it/s]


Processing: exp_12 >> trial_2 >> subject-1 >> 03


...

100%|██████████| 413/413 [00:00<00:00, 301260.44it/s]


Processing: exp_12 >> trial_3 >> subject-1 >> 05


100%|██████████| 414/414 [00:06<00:00, 62.56it/s]


-- Finding optimal global positions for registration


100%|██████████| 414/414 [00:00<00:00, 319551.32it/s]


In [ ]:
def evaluate_adjusted(config: Config):
    if not os.path.exists(config.get_output_file(f"{config.get_file_name()}.npz")):
        print("Unable to find trajectory data. Skipping.")
        return
    
    if not os.path.exists(os.path.join(config.get_groundtruth_dir(), f"{config.get_file_name()}.gtpose.npz")):
        print("Unable to find groundtruth data. Skipping.")
        return
    
    overlap_file = config.get_output_file(f"{config.get_file_name()}__overlap.csv")
    overlap_df = pd.read_csv(overlap_file)

    groundtruth_data = np.load(os.path.join(config.get_groundtruth_dir(), f"{config.get_file_name()}.gtpose.npz"))
    estimated_data = np.load(config.get_output_file(f"{config.get_file_name()}.npz"))
    
    feature_dir = config.get_feature_dir()
    sequence_ts = fread.get_timstamps(feature_dir, ext=".secondary.npz")
    # sequence_ts = fread.sample_timestamps(sequence_ts, config.target_fps)
    
    target_ts = estimated_data["sequence_ts"]
    
    target_inds = [np.argwhere(sequence_ts == target_ts[i])[0][0] for i in range(len(target_ts))]
    
    groundtruth_t = groundtruth_data["local_t"][target_inds]
    estimated_t = estimated_data["global_t"]
    
    # print(len(groundtruth_t), len(estimated_t), groundtruth_t.sum())
    
    # sequence_ts = sequence_ts[target_inds]
    # num_frames = len(sequence_ts)
    
    # groundtruth_data = np.load(os.path.join(config.get_groundtruth_dir(), f"{config.get_file_name()}.gtpose.npz"))
    # estimated_data = np.load(config.get_output_file(f"{config.get_file_name()}.npz"))

    # sequence_ts = fread.get_timstamps(config.get_feature_dir(), ext=".secondary.npz")
    # sequence_ts = fread.sample_timestamps(sequence_ts, config.target_fps)

    # estimated_t = estimated_data["global_t"]
    # groundtruth_t = groundtruth_data["local_t"]

    translation_error = []
    rotation_error = []
    num_samples = 0
    
    for start_t, end_t, overlap, has_enough_global_frames, availability in overlap_df.values:
        
        if overlap < 0.05 or not has_enough_global_frames or availability < 0.5: continue
        
        for t in range(int(start_t), int(end_t)):
            
            if np.sum(estimated_t[t]) == 4: continue
            if np.sum(groundtruth_t[t]) == 0: continue
            
            er, et = transform.calc_error(estimated_t[t], groundtruth_t[t])
            translation_error.append(et)
            rotation_error.append(er)
            num_samples += 1
            
    if len(translation_error) == 0:
        print("No valid fragments found!")
        return [np.nan, np.nan]
        
    print(f"Translation error: {np.mean(translation_error):.3f} ({np.std(translation_error):.3f})", end="\t")
    print(f"Rotation error: {np.mean(rotation_error):.3f} ({np.std(rotation_error):.3f})", end="\t")
    print(f"Num samples: {num_samples}")

    return [np.mean(translation_error), np.mean(rotation_error)]

In [6]:
data = []

for trial in os.listdir(os.path.join(config.feature_dir, config.experiment)):
    config.trial = trial
    for subject in os.listdir(os.path.join(config.feature_dir, config.experiment, config.trial, str(config.voxel_size))):
        config.subject = subject    
        for sequence in os.listdir(os.path.join(config.feature_dir, config.experiment, config.trial, str(config.voxel_size), config.subject)):
            config.sequence = sequence
            print(f"Processing: {config.experiment} >> {config.trial} >> {config.subject} >> {config.sequence}")
            # evaluate_adjusted(config)
            data.append([config.trial, config.subject, config.sequence] + evaluate_adjusted(config))

df = pd.DataFrame(data, columns=["trial", "subject", "sequence", "translation_error", "rotation_error"])
output_path = config.get_output_file("results_summary.csv")

df.to_csv(config.get_output_file(f"evaluation_{config.voxel_size}_{config.target_fps}_adjusted.csv"), index=False)


Processing: exp_12 >> trial_1 >> subject-1 >> 01
Translation error: 2.776 (0.812)	Rotation error: 88.891 (1.011)	Num samples: 126
Processing: exp_12 >> trial_1 >> subject-1 >> 02
Translation error: 0.179 (0.408)	Rotation error: 7.897 (29.247)	Num samples: 167
Processing: exp_12 >> trial_1 >> subject-1 >> 03
Translation error: 0.028 (0.038)	Rotation error: 1.097 (1.483)	Num samples: 195
Processing: exp_12 >> trial_1 >> subject-1 >> 04
Translation error: 0.061 (0.028)	Rotation error: 12.606 (33.575)	Num samples: 281
Processing: exp_12 >> trial_1 >> subject-1 >> 05
Translation error: 0.051 (0.010)	Rotation error: 0.464 (0.259)	Num samples: 160
Processing: exp_12 >> trial_2 >> subject-1 >> 01
Translation error: 1.839 (1.162)	Rotation error: 105.821 (27.877)	Num samples: 173
Processing: exp_12 >> trial_2 >> subject-1 >> 02
No valid fragments found!
Processing: exp_12 >> trial_2 >> subject-1 >> 03
No valid fragments found!
Processing: exp_12 >> trial_2 >> subject-1 >> 04
Translation error: 0

In [2]:
def evaluate_adjusted(config: Config):
    if not os.path.exists(config.get_output_file(f"{config.get_file_name()}.npz")):
        print("Unable to find trajectory data. Skipping.")
        return
    
    if not os.path.exists(os.path.join(config.get_groundtruth_dir(), f"{config.get_file_name()}.gtpose.npz")):
        print("Unable to find groundtruth data. Skipping.")
        return
    
    overlap_file = os.path.join("data/trajectories/trajectory/overlaps", f"{config.get_file_name()}__overlap.csv")
    overlap_df = pd.read_csv(overlap_file)

    groundtruth_data = np.load(os.path.join(config.get_groundtruth_dir(), f"{config.get_file_name()}.gtpose.npz"))
    estimated_data = np.load(config.get_output_file(f"{config.get_file_name()}.npz"))
    
    feature_dir = config.get_feature_dir()
    sequence_ts = fread.get_timstamps(feature_dir, ext=".secondary.npz")
    
    target_ts = estimated_data["sequence_ts"]
    
    target_inds = [np.argwhere(sequence_ts == target_ts[i])[0][0] for i in range(len(target_ts))]
    
    groundtruth_t = groundtruth_data["local_t"][target_inds]
    estimated_t = estimated_data["global_t"]
    
    translation_error = []
    rotation_error = []
    num_samples = 0
    
    for start_t, end_t, overlap, has_enough_global_frames, availability in overlap_df.values:
        
        if overlap < 0.05 or not has_enough_global_frames or availability < 0.5: continue
        
        for t in range(int(start_t), int(end_t)):
            
            if np.sum(estimated_t[t]) == 4: continue
            if np.sum(groundtruth_t[t]) == 0: continue
            
            er, et = transform.calc_error(estimated_t[t], groundtruth_t[t])
            translation_error.append(et)
            rotation_error.append(er)
            num_samples += 1
            
    if len(translation_error) == 0:
        print("No valid fragments found!")
        return [np.nan, np.nan]
        
    print(f"Translation error: {np.mean(translation_error):.3f} ({np.std(translation_error):.3f})", end="\t")
    print(f"Rotation error: {np.mean(rotation_error):.3f} ({np.std(rotation_error):.3f})", end="\t")
    print(f"Num samples: {num_samples}")

    return [np.mean(translation_error), np.mean(rotation_error)]

In [8]:
config = Config(
        sequence_dir="data/raw_data",
        feature_dir="data/features",
        output_dir="",
        experiment="exp_12",
        trial="trial_1",
        subject="subject-1",
        sequence="01",
        groundtruth_dir="data/trajectories/groundtruth",
    )
    
config.voxel_size=0.03
config.target_fps=20
config.min_std=0.5

In [ ]:
method = f"IMU_PCD_{config.voxel_size}"

for i in range(1, 10):
    print(f"Iteration: {i}")

    config.output_dir=f"data/trajectories/trajectory/{i}/{method}"

    data = []

    for trial in os.listdir(os.path.join(config.feature_dir, config.experiment)):
        config.trial = trial
        for subject in os.listdir(os.path.join(config.feature_dir, config.experiment, config.trial, str(config.voxel_size))):
            config.subject = subject    
            for sequence in os.listdir(os.path.join(config.feature_dir, config.experiment, config.trial, str(config.voxel_size), config.subject)):
                config.sequence = sequence
                print(f"Processing: {config.experiment} >> {config.trial} >> {config.subject} >> {config.sequence}")
                # evaluate_adjusted(config)
                data.append([config.trial, config.subject, config.sequence] + evaluate_adjusted(config))

    df = pd.DataFrame(data, columns=["trial", "subject", "sequence", "translation_error", "rotation_error"])
    output_path = config.get_output_file("results_summary.csv")

    df.to_csv(os.path.join("results/trajectory_performance/iterations", f"summary_{method}_{i}.csv"), index=False)

In [7]:
def plot_trajectories(config: Config):
    overlap_file = config.get_output_file(f"{config.get_file_name()}__overlap.csv")
    overlap_df = pd.read_csv(overlap_file)

    sequence_ts = fread.get_timstamps(config.get_feature_dir(), ext=".secondary.npz")
    # sequence_ts = fread.sample_timestamps(sequence_ts, config.target_fps)
    
    groundtruth_data = np.load(os.path.join(config.get_groundtruth_dir(), f"{config.get_file_name()}.gtpose.npz"))
    estimated_data = np.load(config.get_output_file(f"{config.get_file_name()}.npz"))
    
    target_ts = estimated_data["sequence_ts"]

    target_inds = [np.argwhere(sequence_ts == target_ts[i])[0][0] for i in range(len(target_ts))]
    
    groundtruth_t = groundtruth_data["local_t"][target_inds]
    estimated_t = estimated_data["global_t"]
    
    sequence_ts = sequence_ts[target_inds]

    num_frames = len(sequence_ts)

    local_pcds = []

    for t in tqdm.trange(num_frames):
        feature_file = os.path.join(config.get_feature_dir(), f"{sequence_ts[t]}.secondary.npz")
        pcd = FCGF.get_features(feature_file, config.voxel_size, pcd_only=True)
        local_pcds.append(pcd)
        
    target = FCGF.get_features("data/reference/larc_kitchen_v4.npz", config.voxel_size, pcd_only=True)
    target_points = np.asarray(target.points)
    tx, ty = target_points[:, 0], target_points[:, 2]
        
    
    plt.figure(figsize=(20, 10))
    overlap = []

    for i, (start_t, end_t, overlap, has_enough_global_frames, availability) in enumerate(overlap_df.values):
        estimated_fragment = []
        groundtruth_fragment = []

        for t in range(int(start_t), int(end_t)):
            if np.sum(estimated_t[t]) == 4: continue
            if np.sum(groundtruth_t[t]) == 0: continue
            
            local_temp = copy.deepcopy(local_pcds[t])
            local_temp.transform(estimated_t[t])
            estimated_fragment.append(local_temp)
            
            local_temp = copy.deepcopy(local_pcds[t])
            local_temp.transform(groundtruth_t[t])
            groundtruth_fragment.append(local_temp)
            
        estimated_fragment = pointcloud.merge_pcds(estimated_fragment, 0.03)
        groundtruth_fragment = pointcloud.merge_pcds(groundtruth_fragment, 0.03)
        
        estimated_points = np.asarray(estimated_fragment.points)
        ex, ey = estimated_points[:, 0], estimated_points[:, 2]
        
        groundtruth_points = np.asarray(groundtruth_fragment.points)
        gx, gy = groundtruth_points[:, 0], groundtruth_points[:, 2]
        
        plt.subplot(1, len(overlap_df), i + 1)
        plt.scatter(tx, ty, s=0.1, c="r")
        plt.scatter(ex, ey, s=0.1, c="b")
        plt.scatter(gx, gy, s=0.1, c="g")
        plt.xlim(-4.5, 4)
        plt.ylim(-5.5, 4.5)
        plt.gca().set_aspect('equal', adjustable='box')
        plt.axis("off")
        plt.title(f"({start_t}, {end_t}) : {overlap:.2f}, {availability:.2f}, {has_enough_global_frames}")

    plt.savefig(config.get_output_file(f"{config.get_file_name()}__trajectories.jpeg"))
    plt.close()

In [8]:
for trial in os.listdir(os.path.join(config.feature_dir, config.experiment)):
    config.trial = trial
    for subject in os.listdir(os.path.join(config.feature_dir, config.experiment, config.trial, str(config.voxel_size))):
        config.subject = subject    
        for sequence in os.listdir(os.path.join(config.feature_dir, config.experiment, config.trial, str(config.voxel_size), config.subject)):
            config.sequence = sequence
            print(f"Processing: {config.experiment} >> {config.trial} >> {config.subject} >> {config.sequence}")
            plot_trajectories(config)

Processing: exp_12 >> trial_1 >> subject-1 >> 01


100%|██████████| 419/419 [00:06<00:00, 60.65it/s]


Processing: exp_12 >> trial_1 >> subject-1 >> 02


100%|██████████| 412/412 [00:06<00:00, 65.15it/s]


Processing: exp_12 >> trial_1 >> subject-1 >> 03


...

100%|██████████| 416/416 [00:01<00:00, 304.60it/s]


Processing: exp_12 >> trial_3 >> subject-1 >> 04


100%|██████████| 413/413 [00:01<00:00, 332.58it/s]


Processing: exp_12 >> trial_3 >> subject-1 >> 05


100%|██████████| 414/414 [00:01<00:00, 314.64it/s]


In [10]:
methods = ["FPFH", "IMU_PCD"]
voxel_sizes = [0.03, 0.05]
num_iterations = 10

result_dir = "results/trajectory_performance/iterations"

for method in methods:
    for voxel_size in voxel_sizes:
        results = []

        for i in range(1, num_iterations):
            result_file = os.path.join(result_dir, f"summary_{method}_{voxel_size}_{i}.csv")
            results.append(pd.read_csv(result_file))
            

        result = pd.concat(results, ignore_index=True)

        result.dropna(inplace=True)

        result = result.groupby(["trial", "subject", "sequence"])

        result.mean().to_csv(f"results/trajectory_performance/summary_{method}_{voxel_size}_mean.csv")
        result.std().to_csv(f"results/trajectory_performance/summary_{method}_{voxel_size}_std.csv")
        result.min().to_csv(f"results/trajectory_performance/summary_{method}_{voxel_size}_min.csv")

In [25]:
result_dir = "results/trajectory_performance"


for method in methods:
    results = []
    for voxel_size in voxel_sizes:
        result_file = os.path.join(result_dir, f"summary_{method}_{voxel_size}_min.csv")
        results.append(pd.read_csv(result_file))

## Compression 

In [13]:
result_fles = glob.glob("results/compression/global_registration/0.03/*.npz")

In [37]:
m1_count = 0
m2_count = 0
m3_count = 0
m4_count = 0
total_count = 0
total_samples = 0

avg_m1_translation_error = 0
avg_m2_translation_error = 0
avg_m3_translation_error = 0
avg_m4_translation_error = 0

avg_m1_rotation_error = 0
avg_m2_rotation_error = 0
avg_m3_rotation_error = 0
avg_m4_rotation_error = 0

In [38]:
for i in range(len(result_fles)):
    result1 = np.load(result_fles[i])
    result2 = np.load(result_fles[i].replace("0.03", "0.05"))

    translation_error1 = result1["translation_error"]
    rotation_error1 = result1["rotation_error"]
    
    translation_error2 = result2["translation_error"]
    rotation_error2 = result2["rotation_error"]
    
    invalid_inds = np.where(translation_error1[0] == -1)[0]

    translation_error1 = np.delete(translation_error1, invalid_inds, axis=1)
    rotation_error1 = np.delete(rotation_error1, invalid_inds, axis=1)
    
    translation_error2 = np.delete(translation_error2, invalid_inds, axis=1)
    rotation_error2 = np.delete(rotation_error2, invalid_inds, axis=1)

    m1 = np.logical_and(translation_error1[0] < 0.2, rotation_error1[0] < 5)
    m2 = np.logical_and(translation_error1[1] < 0.2, rotation_error1[1] < 5)
    m3 = np.logical_and(translation_error2[0] < 0.2, rotation_error2[0] < 5)
    m4 = np.logical_and(translation_error2[1] < 0.2, rotation_error2[1] < 5)
    
    valid_inds = np.logical_or(m1, np.logical_or(m2, np.logical_or(m3, m4)))
    
    avg_m1_translation_error += translation_error1[0][valid_inds].sum()
    avg_m2_translation_error += translation_error1[1][valid_inds].sum()
    avg_m3_translation_error += translation_error2[0][valid_inds].sum()
    avg_m4_translation_error += translation_error2[1][valid_inds].sum()
    
    avg_m1_rotation_error += rotation_error1[0][valid_inds].sum()
    avg_m2_rotation_error += rotation_error1[1][valid_inds].sum()
    avg_m3_rotation_error += rotation_error2[0][valid_inds].sum()
    avg_m4_rotation_error += rotation_error2[1][valid_inds].sum()
    
    # total_samples += len(translation_error1[0])
    
    # avg_m1_translation_error += translation_error1[0][m1].sum()
    # avg_m2_translation_error += translation_error1[1][m2].sum()
    # avg_m3_translation_error += translation_error2[0][m3].sum()
    # avg_m4_translation_error += translation_error2[1][m4].sum()
    
    # avg_m1_rotation_error += rotation_error1[0][m1].sum()
    # avg_m2_rotation_error += rotation_error1[1][m2].sum()
    # avg_m3_rotation_error += rotation_error2[0][m3].sum()
    # avg_m4_rotation_error += rotation_error2[1][m4].sum()

    # only this one is correct
    # valid_inds = np.logical_or(m1, np.logical_or(m2, np.logical_or(m3, m4)))
    
    # valid_inds = np.logical_or(m1, m2)
    # valid_inds = np.array([1 for _ in range(len(valid_inds))])

    total_count += valid_inds.sum()
    m1_count += m1[valid_inds].sum()
    m2_count += m2[valid_inds].sum()
    m3_count += m3[valid_inds].sum()
    m4_count += m4[valid_inds].sum()

In [39]:
print(f"m1: {m1_count / total_count}")
print(f"m2: {m2_count / total_count}")
print(f"m3: {m3_count / total_count}")
print(f"m4: {m4_count / total_count}")

m1: 0.7348484848484849
m2: 0.6893939393939394
m3: 0.8106060606060606
m4: 0.803030303030303


In [41]:
print(f"Average m1 translation error: {avg_m1_translation_error / m1_count:.3f}")
print(f"Average m2 translation error: {avg_m2_translation_error / m2_count:.3f}")
print(f"Average m3 translation error: {avg_m3_translation_error / m3_count:.3f}")
print(f"Average m4 translation error: {avg_m4_translation_error / m4_count:.3f}")

print(f"Average m1 rotation error: {avg_m1_rotation_error / m1_count:.3f}")
print(f"Average m2 rotation error: {avg_m2_rotation_error / m2_count:.3f}")
print(f"Average m3 rotation error: {avg_m3_rotation_error / m3_count:.3f}")
print(f"Average m4 rotation error: {avg_m4_rotation_error / m4_count:.3f}")

Average m1 translation error: 0.647
Average m2 translation error: 0.915
Average m3 translation error: 0.491
Average m4 translation error: 0.587
Average m1 rotation error: 22.277
Average m2 rotation error: 33.331
Average m3 rotation error: 17.940
Average m4 rotation error: 25.195


In [21]:
print(f"Average m1 translation error: {avg_m1_translation_error / total_samples:.3f}")
print(f"Average m2 translation error: {avg_m2_translation_error / total_samples:.3f}")
print(f"Average m3 translation error: {avg_m3_translation_error / total_samples:.3f}")
print(f"Average m4 translation error: {avg_m4_translation_error / total_samples:.3f}")

print(f"Average m1 rotation error: {avg_m1_rotation_error / total_samples:.3f}")
print(f"Average m2 rotation error: {avg_m2_rotation_error / total_samples:.3f}")
print(f"Average m3 rotation error: {avg_m3_rotation_error / total_samples:.3f}")
print(f"Average m4 rotation error: {avg_m4_rotation_error / total_samples:.3f}")

Average m1 translation error: 1.033
Average m2 translation error: 1.079
Average m3 translation error: 0.933
Average m4 translation error: 0.965
Average m1 rotation error: 54.166
Average m2 rotation error: 56.484
Average m3 rotation error: 54.191
Average m4 rotation error: 54.796
